In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from joblib import Parallel, delayed
jobs = 4

from change_dilation_v2 import change_dilation

In [2]:
# I/O folders
dataset_dir = '../samples/'
outout_dir  = '../results/'

if not os.path.exists(outout_dir):
    os.makedirs(outout_dir)

# Read segmentation data
df = pd.read_csv(dataset_dir + 'segm.csv')
files = list(df["ID"])
data = df.drop(columns=['ID']).to_numpy()

# Output Dilations
dils = np.linspace(0.15, 0.75, 10)

In [3]:
def augment(j):
    # Get output dilation
    dil = dils[j]
    
    # Get output file name
    opt_file_name = '{}_{:02d}.png'.format(ID, j)
    opt_file_name = os.path.join(outout_dir, opt_file_name)
    
    # Change dilation and save
    im2 = change_dilation(im1, dil, pupil_xyr, iris_xyr)
    im2.save(opt_file_name)

In [4]:
for i, name in enumerate(tqdm(files)):
    # Get file name and input image
    ID = name.split('.')[0]
    filename = os.path.join(dataset_dir, name)
    im1 = Image.open(filename)
    
    # Obtain segmentation data
    pupil_xyr = data[i, 0:3]
    iris_xyr  = data[i, 3:6]
    
    # Make data augmentations in parallel
    Parallel(n_jobs = jobs)(
        delayed(augment)(j)
        for j in range(len(dils))
    )
        

100%|████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.83it/s]
